In [1]:
import psycopg2 
import pandas as pd
import numpy as np
import copy

### Loading Data

In [52]:
conn = psycopg2.connect(
    host="fit3164db.cxkhqsoitzhb.ap-southeast-2.rds.amazonaws.com", 
    port=5432,
    user="postgres",
    password="fit3164d13edas",
    database = "testdb"
)
conn.autocommit = True
cursor = conn.cursor()

In [53]:
#actuals columns
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'actuals'")
acc_cols = cursor.fetchall()
acc_cols = [item[0] for item in acc_cols]

cursor.execute("select * from actuals")
actual_data = cursor.fetchall()

In [6]:
#forecasts columns
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'forecasts'")
fc_cols = cursor.fetchall()
fc_cols = [item[0] for item in fc_cols]

cursor.execute("select * from forecasts")
forecast_data = cursor.fetchall()

In [54]:
cursor.close()
conn.close()

In [4]:
actuals = pd.DataFrame(actual_data, columns= acc_cols)
# forecast = pd.DataFrame(forecast_data, columns=fc_cols)

In [55]:
def convert_dtypes(df, actual=True):
    if actual:
        numeric_cols = ['month','hour', 'type_of_day', 'load', 'pressure', 'cloud_cover',  'temperature', 'wind_speed', 'wind_direction']
    else:
        numeric_cols = ['month','hour', 'type_of_day', 'pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction']
    
    date_cols = ['datetime', 'date']
    
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col])
        
    for col in date_cols:
        df[col] = pd.to_datetime(df[col])
    
    return df

actuals = convert_dtypes(actuals)
# forecast = convert_dtypes(forecast, False)

In [56]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [57]:
#lags for load 
def get_lag_lead(data, num_lags=168, forward_pred=48): 
    cols = ['pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction']

    #for load
    for i in range(1, num_lags + 1):
        data[f'load_lag_{i}'] = data['load'].shift(i) 

    #for lagged weather variables
    for c in cols:
        for i in range(1, num_lags+1):
            data[f'{c}_lag_{i}'] = data[c].shift(i)

    for i in range(0, forward_pred+1):    
        data[f'load_lead_{i}'] = data['load'].shift(-i)

    #weather
    for c in cols:
        for i in range(0,forward_pred):
            data[f'{c}_lead_{i}'] = data[c].shift(-i)


    data.dropna(inplace = True)    #drop nulls
    data = data.set_index('datetime')   #set index as datetime
    data = data.asfreq('H')

    data = data.drop(['load', 'pressure', 'wind_speed', 'wind_direction', 'temperature', 'cloud_cover'], axis=1)

    return data

#for validation
def get_lead_load(data, forward_pred=48):
    for i in range(0, forward_pred+1):    
        data[f'load_lead_{i}'] = data['load'].shift(-i)

    return data

#for 
def get_leads(data, i):
    cols = ['pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction']
    
    for c in cols:
        data[f'{c}_lead_{i}'] = data[c].shift(-(i+1))

    return data

In [58]:
data = actuals

In [103]:
data.head()

,hour_sin,hour_cos,month_sin,month_cos,day_sin,day_cos,year,load_lag_1,load_lag_2,load_lag_3,...,wind_direction_lead_38,wind_direction_lead_39,wind_direction_lead_40,wind_direction_lead_41,wind_direction_lead_42,wind_direction_lead_43,wind_direction_lead_44,wind_direction_lead_45,wind_direction_lead_46,wind_direction_lead_47
datetime,,,,,,,,,,,,,,,,,,,,,
2017-03-25 08:00:00,0.866025,-0.5,1.0,6.123234e-17,0.937752,0.347305,2017,1.005723e+06,9.813167e+05,9.760459e+05,...,109.0,215.0,320.0,231.0,142.0,52.0,82.0,113.0,143.0,186.0
2017-03-26 08:00:00,0.866025,-0.5,1.0,6.123234e-17,0.988468,0.151428,2017,9.794722e+05,9.801709e+05,9.894177e+05,...,184.0,179.0,174.0,179.0,184.0,189.0,180.0,172.0,163.0,206.0
2017-03-27 08:00:00,0.866025,-0.5,1.0,6.123234e-17,0.201299,0.979530,2017,1.055779e+06,9.885602e+05,9.583690e+05,...,178.0,179.0,181.0,183.0,184.0,186.0,160.0,134.0,108.0,149.0
2017-03-28 08:00:00,0.866025,-0.5,1.0,6.123234e-17,0.394356,0.918958,2017,1.087711e+06,1.024927e+06,1.008084e+06,...,212.0,227.0,241.0,191.0,140.0,90.0,78.0,67.0,55.0,132.0
2017-03-29 08:00:00,0.866025,-0.5,1.0,6.123234e-17,0.571268,0.820763,2017,1.096427e+06,1.036765e+06,1.017226e+06,...,190.0,190.0,191.0,169.0,147.0,125.0,126.0,126.0,126.0,164.0


In [60]:
# data = pd.merge(actuals[['datetime','load']], forecast, on='datetime')

encode(data, 'hour', 24)
encode(data, 'month', 12)
encode(data, 'type_of_day', 31)

# data = data.set_index('datetime')
data = data.drop(['id', 'date','month', 'hour', 'type_of_day'], axis=1)
data = data.rename(columns = {'type_of_day_sin': 'day_sin', 
                            'type_of_day_cos': 'day_cos'})

In [63]:
data['year']= data['datetime'].apply(lambda x: x.year)

In [64]:
data = get_lag_lead(data)

/var/folders/34/n9z2vp496dx5gwr2m5_w4p780000gn/T/ipykernel_35172/886215247.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'load_lag_{i}'] = data['load'].shift(i)
/var/folders/34/n9z2vp496dx5gwr2m5_w4p780000gn/T/ipykernel_35172/886215247.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'load_lag_{i}'] = data['load'].shift(i)
/var/folders/34/n9z2vp496dx5gwr2m5_w4p780000gn/T/ipykernel_35172/886215247.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` m

/var/folders/34/n9z2vp496dx5gwr2m5_w4p780000gn/T/ipykernel_35172/886215247.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{c}_lag_{i}'] = data[c].shift(i)
/var/folders/34/n9z2vp496dx5gwr2m5_w4p780000gn/T/ipykernel_35172/886215247.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{c}_lag_{i}'] = data[c].shift(i)
/var/folders/34/n9z2vp496dx5gwr2m5_w4p780000gn/T/ipykernel_35172/886215247.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times

In [72]:
desired_start_time = '08:00:00'
desired_end_time = '08:00:00'
start_datetime = pd.to_datetime(desired_start_time)
end_datetime = pd.to_datetime(desired_end_time)
data = data.between_time(start_datetime.time(), end_datetime.time())

In [73]:
data

,hour_sin,hour_cos,month_sin,month_cos,day_sin,day_cos,year,load_lag_1,load_lag_2,load_lag_3,...,wind_direction_lead_38,wind_direction_lead_39,wind_direction_lead_40,wind_direction_lead_41,wind_direction_lead_42,wind_direction_lead_43,wind_direction_lead_44,wind_direction_lead_45,wind_direction_lead_46,wind_direction_lead_47
datetime,,,,,,,,,,,,,,,,,,,,,
2017-03-25 08:00:00,0.866025,-0.5,1.0,6.123234e-17,0.937752,0.347305,2017,1.005723e+06,9.813167e+05,9.760459e+05,...,109.0,215.0,320.0,231.0,142.0,52.0,82.0,113.0,143.0,186.0
2017-03-26 08:00:00,0.866025,-0.5,1.0,6.123234e-17,0.988468,0.151428,2017,9.794722e+05,9.801709e+05,9.894177e+05,...,184.0,179.0,174.0,179.0,184.0,189.0,180.0,172.0,163.0,206.0
2017-03-27 08:00:00,0.866025,-0.5,1.0,6.123234e-17,0.201299,0.979530,2017,1.055779e+06,9.885602e+05,9.583690e+05,...,178.0,179.0,181.0,183.0,184.0,186.0,160.0,134.0,108.0,149.0
2017-03-28 08:00:00,0.866025,-0.5,1.0,6.123234e-17,0.394356,0.918958,2017,1.087711e+06,1.024927e+06,1.008084e+06,...,212.0,227.0,241.0,191.0,140.0,90.0,78.0,67.0,55.0,132.0
2017-03-29 08:00:00,0.866025,-0.5,1.0,6.123234e-17,0.571268,0.820763,2017,1.096427e+06,1.036765e+06,1.017226e+06,...,190.0,190.0,191.0,169.0,147.0,125.0,126.0,126.0,126.0,164.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-08 08:00:00,0.866025,-0.5,0.5,8.660254e-01,0.848644,0.528964,2021,1.046518e+06,1.000318e+06,9.849659e+05,...,333.0,340.0,347.0,350.0,353.0,357.0,319.0,282.0,245.0,205.0
2021-01-09 08:00:00,0.866025,-0.5,0.5,8.660254e-01,0.937752,0.347305,2021,1.018170e+06,9.924154e+05,9.832454e+05,...,346.0,350.0,354.0,317.0,280.0,242.0,165.0,89.0,12.0,11.0
2021-01-10 08:00:00,0.866025,-0.5,0.5,8.660254e-01,0.988468,0.151428,2021,9.924074e+05,9.818259e+05,9.750069e+05,...,230.0,118.0,5.0,6.0,6.0,7.0,7.0,7.0,8.0,48.0


In [74]:
def split_data(data, test_size=0.1):
    return np.split(data, [int((1 - test_size) * data.shape[0]) + 1])

In [75]:
train, validation = split_data(data)

original load, pressure, temp, ... are as load_lead_0, pressure_lead_0, temperature_lead_0 etc.

In [77]:
#get specific lead forecast column names from whole df for each model
def get_lead_forecast_names(i):
    cols = ['pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction']
    lead_cols = []
    for c in cols: 
        lead_cols.append(f'{c}_lead_{i}')
    return lead_cols


def x_cols(i):
    # Specify the path to your text file
    file_path = "train_cols.txt"

    # Initialize an empty list to store the words
    word_list = []

    # Open the file and read its contents line by line
    with open(file_path, 'r') as file:
        for line in file:
            # Remove leading and trailing whitespace and append the word to the list
            word_list.append(line.strip())

    word_list += get_lead_forecast_names(i)

    return word_list


def y_cols(i):
    return f'load_lead_{i}'




In [20]:
from sklearn.linear_model import LinearRegression

In [78]:
NUM_OF_MODELS = 48

In [79]:
import xgboost as xgb

In [106]:
# models = [LinearRegression() for _ in range(NUM_OF_MODELS)]

In [80]:
models = [xgb.XGBRegressor() for _ in range(NUM_OF_MODELS)]

In [96]:
aes, apes, ses = [], [], []

- get lag load and weather for train and get lead weather for specific hour for train x and valid x
- get load for train y
- get lead load for valid y

In [82]:
# fitting
for i in range(NUM_OF_MODELS):
    x_columns = x_cols(i)
    y_columns = y_cols(i)
    
    train_x = train[x_columns]
    train_y = train[y_columns]
 
    current_model = models[i]
    current_model.fit(train_x, train_y)

In [102]:
validation

,hour_sin,hour_cos,month_sin,month_cos,day_sin,day_cos,year,load_lag_1,load_lag_2,load_lag_3,...,wind_direction_lead_38,wind_direction_lead_39,wind_direction_lead_40,wind_direction_lead_41,wind_direction_lead_42,wind_direction_lead_43,wind_direction_lead_44,wind_direction_lead_45,wind_direction_lead_46,wind_direction_lead_47
datetime,,,,,,,,,,,,,,,,,,,,,
2020-08-28 08:00:00,0.866025,-0.5,-0.866025,-5.000000e-01,0.848644,0.528964,2020,1.040583e+06,1.004993e+06,1.003661e+06,...,221.0,236.0,250.0,246.0,243.0,239.0,237.0,234.0,232.0,233.0
2020-08-29 08:00:00,0.866025,-0.5,-0.866025,-5.000000e-01,0.937752,0.347305,2020,9.895095e+05,9.658296e+05,9.683778e+05,...,254.0,255.0,256.0,259.0,262.0,264.0,254.0,245.0,235.0,222.0
2020-08-30 08:00:00,0.866025,-0.5,-0.866025,-5.000000e-01,0.988468,0.151428,2020,9.549121e+05,9.499653e+05,9.561282e+05,...,220.0,212.0,204.0,192.0,179.0,166.0,162.0,159.0,155.0,160.0
2020-08-31 08:00:00,0.866025,-0.5,-0.866025,-5.000000e-01,0.201299,0.979530,2020,1.008517e+06,9.633588e+05,9.589822e+05,...,173.0,125.0,76.0,94.0,113.0,132.0,140.0,147.0,155.0,166.0
2020-09-01 08:00:00,0.866025,-0.5,-1.000000,-1.836970e-16,0.394356,0.918958,2020,1.015909e+06,9.741481e+05,9.692347e+05,...,313.0,318.0,323.0,319.0,315.0,311.0,307.0,303.0,299.0,290.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-08 08:00:00,0.866025,-0.5,0.500000,8.660254e-01,0.848644,0.528964,2021,1.046518e+06,1.000318e+06,9.849659e+05,...,333.0,340.0,347.0,350.0,353.0,357.0,319.0,282.0,245.0,205.0
2021-01-09 08:00:00,0.866025,-0.5,0.500000,8.660254e-01,0.937752,0.347305,2021,1.018170e+06,9.924154e+05,9.832454e+05,...,346.0,350.0,354.0,317.0,280.0,242.0,165.0,89.0,12.0,11.0
2021-01-10 08:00:00,0.866025,-0.5,0.500000,8.660254e-01,0.988468,0.151428,2021,9.924074e+05,9.818259e+05,9.750069e+05,...,230.0,118.0,5.0,6.0,6.0,7.0,7.0,7.0,8.0,48.0


In [98]:
#prediction
for i in range(NUM_OF_MODELS):

    x_columns = x_cols(i)
    y_columns = y_cols(i)

    valid_x = validation[x_columns]
    valid_y = validation[y_columns].values

    current_model = models[i]
    prediction = current_model.predict(valid_x)

    aes += list(absolute_errors(valid_y, prediction))
    apes += list(absolute_perc_errors(valid_y, prediction))
    ses += list((squared_errors(valid_y, prediction)))



ae: [ 8786.943  33165.124   6675.2366 11573.779   8716.423   7442.702
 10447.173   4804.41    6629.826    346.1469  6234.961   6699.025
 15930.405   2798.361  14373.327  11740.1025  9615.8169  6335.181
  4695.442    653.535   1852.773  18169.73   18258.206  23773.6259
  6413.347  11159.803   3389.553   5854.021    428.241  20021.59
  6759.0485  3397.097  44312.854  30045.095   4391.805  11997.336
 24728.669  10387.0134  3511.775    697.992  30095.467   2378.613
  3420.621   4173.06    2084.2019  8306.165  10432.494  10015.153
  8499.968    517.793   9708.401    992.3469  8014.335    206.508
  4046.438  12542.387   5454.647   4121.995   2468.3859  4811.734
  8341.481   3929.163   5046.89   12937.565  34216.5624 16616.7894
 37874.4149 13522.204   7617.325   2538.372   5564.682  18852.8695
  5263.682  43458.682   5133.693  11780.443   4417.318   5510.307
 12398.432   9271.068  15483.818    326.068   5527.443   9881.193
 14675.193  15846.818   9347.068  16195.193  13867.818   8864.568
 362

ae: [5.64325660e+04 2.18362150e+04 2.20058534e+04 2.94278620e+04
 4.45827100e+03 5.64588270e+04 4.71989520e+04 3.77876510e+04
 1.08566390e+04 1.32555825e+04 1.80152700e+04 5.39043300e+03
 7.95996770e+04 2.82749600e+03 1.46505770e+04 1.23452100e+04
 1.04457155e+04 1.08587700e+03 2.47344140e+04 2.21567520e+04
 2.93518600e+03 5.06223570e+04 4.09757420e+04 2.50730760e+04
 4.67375700e+04 3.65759710e+04 2.93209020e+04 2.14879560e+04
 4.86296350e+04 2.85696640e+04 3.60130910e+04 5.60785700e+04
 2.89911540e+04 4.22266000e+04 2.58975910e+04 7.04872500e+03
 3.97841000e+03 7.47611050e+03 4.29544900e+03 1.43131850e+04
 9.26488800e+03 8.68179900e+03 4.38190400e+04 2.55817860e+04
 1.62366090e+04 1.38283420e+04 1.70153810e+04 2.62237760e+04
 2.66580370e+04 4.26220910e+04 2.98975950e+04 3.04087500e+03
 1.96251330e+04 8.97555700e+03 2.53473530e+04 4.38002940e+04
 1.44962190e+04 1.51021120e+04 1.55634840e+04 5.65441700e+03
 2.73809510e+04 2.33520300e+04 2.05405980e+04 1.66495350e+04
 4.30126090e+04 4.66

ae: [  7732.7909   6410.8701    142.7814  12960.5184  27140.1731   5427.1731
   1216.5546  13195.7014  12190.4501  21254.3989    771.5944  49560.1485
  31769.8784  12401.3606  23084.7029   1212.6104   4210.1511  14744.1401
   8086.3624   1312.1704  53542.4945   8007.1519   7839.6056   1841.5724
   4908.1199   2341.8836  15213.9286  10534.3476  31446.007    2072.4225
   2569.7401 103048.6384  20497.9794  38899.7631   1414.3596  15657.8586
   7383.0386   4201.8581  13862.4926  27324.4716  16343.3619   3087.3674
   7177.0486  24381.9844   1450.3279  11392.8951   2145.4799   1424.0209
   6604.3911  12854.9216   8408.4604   9029.4579  18692.8076   5564.1936
   9014.2471   6803.8029   5329.0064  15754.9704   9960.8604  12476.8701
  12518.6554   9739.9366   6953.9699  37994.0101  11379.4004  23025.3446
   1238.7846  15410.9461  10390.8026  11488.9945   1297.3695  15694.7445
  13522.1305   2440.2555  32351.9945   2122.318   28635.307     830.557
  24733.3695   1717.932   31634.3695  15542.3805

ae: [ 36793.733   48975.16    22731.251    2417.116   58442.952   54057.077
  55313.637   43903.607    8036.189  121345.101   83270.006   23293.751
  14292.637    4787.923   70539.715   14231.08    48070.498   62977.444
   4309.892   23995.308  172698.923   21923.35    37327.021   62282.955
 102850.702   68825.256    5250.649  100089.486   26875.895    5599.85
  35266.705  100503.993    1634.493   37951.681     532.222   52298.62
  39593.643   12220.787    2994.492   64928.853  100149.151    1503.724
  63174.731   62765.834   51531.604   14488.983   16362.721   40917.542
  58319.558   10653.904    4475.944    6005.569   20006.231   24917.059
  43462.239    8162.602   17553.554   43017.54     9713.537   22102.105
  75881.794   40052.157   78131.854  114860.182   39818.261  212977.872
   5679.478   51371.613   16349.274   49445.057   22796.307    3651.057
 137101.932   41046.818   26816.568   42633.568   26740.568   40167.568
  10204.682   55407.568   49149.932   25888.932    8022.932   

ae: [  6113.022     939.6524  26944.8249  30615.298   42650.6105  14783.8085
   3642.0825  17155.2505  20873.1806   3624.3815  39918.39    41086.1165
  12183.298   14524.6819  11086.161   14892.0839  19179.6005   9541.121
   8345.35    45390.6065  13560.017   36365.729   31813.157   20800.3335
   4289.2885  11809.5315  12931.073   31117.6985   6719.522    5446.0945
 116310.3019  50058.7994  13019.3104  16622.7145  26634.2221  12089.1026
  13136.5844  27424.9725  59893.9215  17499.085   65474.7059  23502.5125
  13389.4809  27507.5829  16409.1645   9938.596   23199.905   13021.275
  13235.482    1932.4375  32387.6365  25155.544   12608.6915   4272.093
    137.6385  16829.631     946.6645   5376.4095  24034.424   14630.4404
  22474.9499  28192.9759  20666.5194  16005.8946   4687.8621   6303.533
  41591.1535  36807.732    7299.1305  28233.9945  22817.932    2738.807
   4076.6195  30752.932   21636.4945   2023.068   11792.9945  13751.7445
  21423.3695  34012.182   41747.307    7385.182   10

In [47]:
preds = models[0].predict(validation[x_cols(0)])
valid_y = validation[y_cols(0)].values

In [99]:
print(get_mae(aes))

23880.997317149755


In [100]:
get_rmse(ses)

34876.94582163186

In [101]:
get_mape(apes)

2.155213783723934

In [92]:
def absolute_errors(actuals, predicted):
    ae = np.abs(np.array(actuals)-np.array(predicted))
    print(f"ae: {ae}")
    return ae

def squared_errors(actuals, predicted):
    se = np.square(np.array(actuals)-np.array(predicted) )
    print(f"se: {se}")
    return se

def absolute_perc_errors(actuals, predicted):
    ape = ((np.abs(np.array(actuals)-np.array(predicted)))/ np.array(actuals)) * 100
    print(f"ape: {ape}")
    return ape

def get_mae(aes):
    return np.mean(aes)

def get_rmse(ses):
    return np.sqrt(np.mean(ses))

def get_mape(apes):
    return np.mean(apes)

In [ ]:
#prediction
for i in range(NUM_OF_MODELS):

    x_columns = x_cols(i)
    y_columns = y_cols(i)

    valid_x = validation[x_columns]
    valid_y = validation[y_columns].values

    current_model = models[i]
    prediction = current_model.predict(valid_x)

    aes += list(absolute_errors(valid_y, prediction))
    apes += list(absolute_perc_errors(valid_y, prediction))
    ses += list((squared_errors(valid_y, prediction)))
